In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as func


In [2]:
spark=SparkSession.builder.master("local").appName("sql_df").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
read_csv=spark.read.csv(r"C:\Spark Course\SQL-fakefriends\fakefriends.csv",header=False,inferSchema="true")
read_csv = read_csv.withColumnRenamed("_c0", "id")
read_csv = read_csv.withColumnRenamed("_c1", "name")
read_csv = read_csv.withColumnRenamed("_c2", "age")
read_csv = read_csv.withColumnRenamed("_c3", "friends-num")

In [3]:
read_csv

id,name,age,friends-num
0,Will,33,385
1,Jean-Luc,26,2
2,Hugh,55,221
3,Deanna,40,465
4,Quark,68,21
5,Weyoun,59,318
6,Gowron,37,220
7,Will,54,307
8,Jadzia,38,380
9,Hugh,27,181


In [4]:
read_csv.createOrReplaceTempView("table1")
query1=spark.sql("select * from table1 where age<=19 and age>=13 ")
query1.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends-num: integer (nullable = true)



In [5]:
query1.show()

+---+-------+---+-----------+
| id|   name|age|friends-num|
+---+-------+---+-----------+
| 21|  Miles| 19|        268|
| 52|Beverly| 19|        269|
| 54|  Brunt| 19|          5|
|106|Beverly| 18|        499|
|115|  Dukat| 18|        397|
|133|  Quark| 19|        265|
|136|   Will| 19|        335|
|225|   Elim| 19|        106|
|304|   Will| 19|        404|
|341|   Data| 18|        326|
|366|  Keiko| 19|        119|
|373|  Quark| 19|        272|
|377|Beverly| 18|        418|
|404| Kasidy| 18|         24|
|409|    Nog| 19|        267|
|439|   Data| 18|        417|
|444|  Keiko| 18|        472|
|492|  Dukat| 19|         36|
|494| Kasidy| 18|        194|
+---+-------+---+-----------+



In [6]:
read_csv.select("age").show()


+---+
|age|
+---+
| 33|
| 26|
| 55|
| 40|
| 68|
| 59|
| 37|
| 54|
| 38|
| 27|
| 53|
| 57|
| 54|
| 56|
| 43|
| 36|
| 22|
| 35|
| 45|
| 60|
+---+
only showing top 20 rows



In [7]:
read_csv.filter(read_csv.age<19).show()

+---+-------+---+-----------+
| id|   name|age|friends-num|
+---+-------+---+-----------+
|106|Beverly| 18|        499|
|115|  Dukat| 18|        397|
|341|   Data| 18|        326|
|377|Beverly| 18|        418|
|404| Kasidy| 18|         24|
|439|   Data| 18|        417|
|444|  Keiko| 18|        472|
|494| Kasidy| 18|        194|
+---+-------+---+-----------+



In [8]:
read_csv.select("name").show()

+--------+
|    name|
+--------+
|    Will|
|Jean-Luc|
|    Hugh|
|  Deanna|
|   Quark|
|  Weyoun|
|  Gowron|
|    Will|
|  Jadzia|
|    Hugh|
|     Odo|
|     Ben|
|   Keiko|
|Jean-Luc|
|    Hugh|
|     Rom|
|  Weyoun|
|     Odo|
|Jean-Luc|
|  Geordi|
+--------+
only showing top 20 rows



In [9]:
read_csv.groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
| 34|    6|
| 28|   10|
| 26|   17|
| 27|    8|
| 44|   12|
| 22|    7|
| 47|    9|
| 52|   11|
| 40|   17|
| 20|    5|
| 57|   12|
| 54|   13|
| 48|   10|
| 19|   11|
| 64|   12|
| 41|    9|
| 43|    7|
+---+-----+
only showing top 20 rows



In [10]:
read_csv.groupBy("age").avg("friends-num").sort("age").show()

+---+------------------+
|age|  avg(friends-num)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



In [11]:
read_csv.groupBy("age").agg(func.round(func.avg("friends-num"),2).alias("avg123")).sort("age").show()

+---+------+
|age|avg123|
+---+------+
| 18|343.38|
| 19|213.27|
| 20| 165.0|
| 21|350.88|
| 22|206.43|
| 23| 246.3|
| 24| 233.8|
| 25|197.45|
| 26|242.06|
| 27|228.13|
| 28| 209.1|
| 29|215.92|
| 30|235.82|
| 31|267.25|
| 32|207.91|
| 33|325.33|
| 34| 245.5|
| 35|211.63|
| 36| 246.6|
| 37|249.33|
+---+------+
only showing top 20 rows



In [12]:
spark.stop()